In [ ]:
import requests
import pandas as pd
import csv
import time

BASE_URL = "https://api.semanticscholar.org/graph/"
VERSION = "v1/"
RESOURCE = "author/search"
my_url = BASE_URL + VERSION + RESOURCE

# util
def most_frequent(List):
    return max(set(List), key = List.count)

In [ ]:
data = []
for name in all_names:
    params = {
        "query": name,
        "limit": 1,
        "fields": "name,aliases,authorId,papers.title,papers.authors,papers.abstract,papers.year,papers.externalIds,papers.s2FieldsOfStudy,papers.citationCount"}
    success = False
    while (not success):
        r = requests.get(my_url, params=params)
        if (r.status_code != 200):
            time.sleep(4)
        else:
            data.append(r.json()['data'])
            success = True

In [ ]:
# AUTHORS DATA
name_list = []
aliases_list = []
authorId_list = []
authorCitationCount_list = []
authorField_list = []
authorFrequentField_list = []
papers_list = []

# PAPERS DATA
paperId_list = []
title_list = []
year_list = []
abstract_list = []
externalIdDOI_list = []
paperCitationCount_list = []
paperFields_list = []
paperAuthorIds_list = []

for d in data:
    if (len(d) == 0):
        continue
    d = d[0]

    # AUTHORS DATA
    authorCitationCounter = 0
    name = d.get('name')
    aliases = d.get('aliases')
    authorId = d.get('authorId')
    papers = d.get('papers')

    name_list.append(name)
    aliases_list.append(aliases)
    authorId_list.append(authorId)
    papers_list.append(papers)

    # PAPERS DATA
    for paper in papers_list:
        paper = paper[0]

        try:
            year = int(paper.get('year'))
            citationCount = int(paper.get('citationCount'))
            authorCitationCounter += citationCount
        except:
            continue
        paperId = paper.get('paperId')
        title = paper.get('title')
        authors_raw = paper.get('authors')
        abstract = paper.get('abstract')
        externalIdsDOI = paper.get('externalIds').get('DOI')
        s2FieldsOfStudy = paper.get('s2FieldsOfStudy')

    paperId_list.append(paperId)
    title_list.append(title)
    year_list.append(year)
    abstract_list.append(abstract)
    externalIdDOI_list.append(externalIdsDOI)
    paperCitationCount_list.append(citationCount)
    
    fields = []
    authors = []
    for field in s2FieldsOfStudy:
        fields.append(field.get('category'))
        authorField_list.append(field.get('category'))
    for author in authors_raw:
        authors.append(field.get('authorId'))
    paperFields_list.append(fields)
    paperAuthorIds_list.append(authors)

    authorCitationCount_list.append(authorCitationCounter)
    authorFrequentField_list.append(most_frequent(authorField_list))

authors_df = pd.DataFrame({'authorId': authorId_list, 'name': name_list, 'aliases': aliases_list, 'citationCount': authorCitationCount_list, 'field': authorFrequentField_list})
papers_df = pd.DataFrame({'paperId': paperId_list, 'title': title_list, 'year': year_list, 'externalIdDOI': externalIdDOI_list, 'citationCount': paperCitationCount_list, 'fields': paperFields_list, 'authorIds': paperAuthorIds_list})
paper_abstract_df = pd.DataFrame({'paperId': paperId_list, 'abstract': abstract_list})

In [ ]:
authors_df.to_csv(path_or_buf='./data/authors.csv')
papers_df.to_csv(path_or_buf='./data/papers.csv')
paper_abstract_df.to_csv(path_or_buf='./data/abstracts.csv')